In [1]:
%%time

from premise import *
import bw2data
bw2data.projects.set_current("ecoinvent-3.10-cutoff")
#bw2data.projects.set_current("ecoinvent-3.9.1-cutoff_bw25")
# clear_inventory_cache()
scenarios = [
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2020},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2030},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2040},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2050},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2060},
    #{"model": "remind", "pathway": "SSP2-PkBudg1150", "year": 2020},
    {"model": "witch", "pathway": "SSP2-Base", "year": 2050},
    {"model": "witch", "pathway": "SSP2-PkBudg1150", "year": 2050},
    {"model": "image", "pathway": "SSP2-RCP26", "year": 2030},
    {"model": "remind", "pathway": "SSP2-PkBudg1150", "year": 2040},
    {"model": "tiam-ucl", "pathway": "SSP2-RCP26", "year": 2040},

    #{"model": "remind", "pathway": "SSP2-PkBudg1150", "year": 2060},
]

ndb = NewDatabase(
        scenarios=scenarios,
        source_db="ecoinvent-3.10.1-cutoff",
        source_version="3.10",
        biosphere_name="biosphere3",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
)
#ndb.write_db_to_brightway("test final energy 2")

premise v.(2, 3, 0, 'dev0')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-----

In [2]:
ndb.generate_scenario_report()

Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.


In [2]:
ndb.update()

Processing scenarios for all sectors:   0%|     | 0/1 [00:00<?, ?it/s]

No cement scenario data available -- skipping
No steel scenario data available -- skipping
No residential heat scenario data available -- skipping
No DAC scenario data available -- skipping
No EWR scenario data available -- skipping
No CDR scenario data available -- skipping
No two-wheeler fleet scenario data available -- skipping
No bus fleet scenario data available -- skipping
No train fleet scenario data available -- skipping
No final energy scenario data available -- skipping


Processing scenarios for all sectors: 100%|█| 1/1 [02:51<00:00, 171.76

Done!



In [3]:
ndb.write_db_to_brightway("witch test 3")

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.
Title: Writing activities to SQLite3 database:
  Started: 04/09/2025 09:37:43
  Finished: 04/09/2025 09:38:12
  Total time elapsed: 00:00:29
  CPU %: 89.40
  Memory %: 16.14
Created database: witch test 3
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev.


In [6]:
ndb.scenarios[0]["iam data"].petrol_efficiencies.sel(region="eu27", year=2050)

AttributeError: 'IAMDataCollection' object has no attribute 'petrol_efficiencies'

In [ ]:
from premise.utils import load_database

for scenario in ndb.scenarios:
    scenario = load_database(scenario)

In [ ]:
for ds in ndb.scenarios[0]["database"]:
    if ds["name"] == "heat, residential, by combustion of methanol using boiler, distributed by truck, produced with biomass" and ds["location"] == "brazil":
        for e in ds["exchanges"]:
            print(e["name"], e["amount"])
        print()
        break

In [ ]:
ndb.scenarios[0]["iam data"].passenger_car_efficiencies.sel(region="india")

In [ ]:
expected_co2 = 0.0

energy = sum(
    [
        exc["amount"]
        for exc in ds["exchanges"]
        if exc["unit"] == "megajoule" and exc["type"] == "technosphere"
    ]
)
# add input of coal
coal = sum(
    [
        exc["amount"] * 26.4
        for exc in ds["exchanges"]
        if "hard coal" in exc["name"]
        and exc["type"] == "technosphere"
        and exc["unit"] == "kilogram"
    ]
)

expected_co2 += coal * 0.098
print(expected_co2)

# add input of coal briquettes
briquettes = sum(
    [
        exc["amount"]
        for exc in ds["exchanges"]
        if "briquettes" in exc["name"]
        and exc["type"] == "technosphere"
        and exc["unit"] == "megajoule"
    ]
)

expected_co2 += briquettes * 0.098
print(expected_co2)

# add input of natural gas
nat_gas = sum(
    [
        exc["amount"] * (36 if exc["unit"] == "cubic meter" else 47.5)
        for exc in ds["exchanges"]
        if "natural gas" in exc["name"]
        and exc["type"] == "technosphere"
        and exc["unit"] in ["cubic meter", "kilogram"]
    ]
)

expected_co2 += nat_gas * 0.06
print(expected_co2)

# add input of liquefied natural gas

lpg = sum(
    [
        exc["amount"] * (36 if exc["unit"] == "cubic meter" else 47.5)
        for exc in ds["exchanges"]
        if "liquefied petroleum gas" in exc["name"]
        and exc["type"] == "technosphere"
        and exc["unit"] in ["cubic meter", "kilogram"]
    ]
)

expected_co2 += lpg * 0.0631
print(expected_co2)

# add input of diesel
diesel = sum(
    [
        exc["amount"] * 42.6
        for exc in ds["exchanges"]
        if "diesel" in exc["name"]
        and exc["type"] == "technosphere"
        and exc["unit"] == "kilogram"
    ]
)

expected_co2 += diesel * 0.0732
print(expected_co2)

# add input of light fuel oil
light_fue_oil = sum(
    [
        exc["amount"] * 41.8
        for exc in ds["exchanges"]
        if "light fuel oil" in exc["name"]
        and exc["type"] == "technosphere"
        and exc["unit"] == "kilogram"
    ]
)

expected_co2 += light_fue_oil * 0.0686
print(expected_co2)

# add input of heavy fuel oil
heavy_fuel_oil = sum(
    [
        exc["amount"] * 41.8
        for exc in ds["exchanges"]
        if "heavy fuel oil" in exc["name"]
        and exc["type"] == "technosphere"
        and exc["unit"] == "kilogram"
    ]
)

expected_co2 += heavy_fuel_oil * 0.0739
print(expected_co2)

# add input of biomass
biomass = sum(
    [
        exc["amount"] * 18
        for exc in ds["exchanges"]
        if any(x in exc["name"] for x in ["biomass", "wood", "timber"])
        and exc["type"] == "technosphere"
        and exc["unit"] == "kilogram"
    ]
)

expected_co2 += biomass * 0.112
print(expected_co2)

# add input of methane
methane = sum(
    [
        exc["amount"] * (36 if exc["unit"] == "cubic meter" else 47.5)
        for exc in ds["exchanges"]
        if "methane" in exc["name"]
        and exc["type"] == "technosphere"
        and exc["unit"] in ["kilogram", "cubic meter"]
    ]
)

expected_co2 += methane * 0.06
print(expected_co2)

# add input of biogas
biogas = sum(
    [
        exc["amount"] * 22.7
        for exc in ds["exchanges"]
        if "biogas" in exc["name"]
        and exc["type"] == "technosphere"
        and exc["unit"] == "cubic meter"
    ]
)

expected_co2 += biogas * 0.058
print(expected_co2)

# add input of methanol
methanol = sum(
    [
        exc["amount"] * 20
        for exc in ds["exchanges"]
        if "methanol" in exc["name"]
        and exc["type"] == "technosphere"
        and exc["unit"] == "kilogram"
    ]
)

expected_co2 += methanol * 0.069
print(expected_co2)

# add input of hydrogen
hydrogen = sum(
    [
        exc["amount"] * 120
        for exc in ds["exchanges"]
        if "hydrogen" in exc["name"]
        and exc["type"] == "technosphere"
        and exc["unit"] == "kilogram"
    ]
)

# add input of electricity
electricity = sum(
    [
        exc["amount"] * 3.6
        for exc in ds["exchanges"]
        if "electricity" in exc["name"]
        and exc["type"] == "technosphere"
        and exc["unit"] == "kilowatt hour"
    ]
)

energy_input = sum(
    [
        energy,
        coal,
        briquettes,
        nat_gas,
        lpg,
        diesel,
        light_fue_oil,
        heavy_fuel_oil,
        biomass,
        methane,
        biogas,
        hydrogen,
        methanol,
        electricity,
    ]
)

In [ ]:
[
        exc
        for exc in ds["exchanges"]
        if any(x in exc["name"] for x in ["biomass", "wood", "timber"])
        and exc["type"] == "technosphere"
        and exc["unit"] == "kilogram"
    ]

In [ ]:
ndb.scenarios[0]["iam data"].biomass_mix.sel(region="canada", year=2050)

In [ ]:
bw2data.projects.set_current("ecoinvent-3.9.1-cutoff_bw25")
bw2data.databases

In [ ]:
ndb.generate_scenario_report()

In [ ]:
ndb.scenarios[0]["iam data"].daccs_energy_use

In [ ]:
ndb.update([
    #"fuels",
    "heat",
    "final energy"
])

In [ ]:
ndb.write_db_to_brightway("heat test 1")

In [ ]:
ndb.write_superstructure_db_to_brightway("heat test 1")

In [ ]:
from premise.utils import load_database

for scenario in ndb.scenarios:
    scenario = load_database(scenario)


for scenario in ndb.scenarios:
    for ds in scenario["database"]:
        if ds["name"] == "biodiesel, burned in passenger train" and ds["location"] == "LAM":
            for e in ds["exchanges"]:
                    print(e["name"], e["amount"])

In [ ]:
ndb.scenarios[0]["iam data"].residential_heating_mix.sel(region="CAZ")

In [ ]:
ndb.update(
    [
        #"electricity",
        #"dac",
        #"fuels",
        "heat",
        #"final energy"
    ]
)

In [ ]:
ndb.generate_scenario_report()

In [ ]:
ndb.write_db_to_brightway("test final energy 12")

In [ ]:
import copy
import json
import ujson
import pickle

In [ ]:
%%time

data = pickle.dumps(ndb.database, -1)
db = pickle.loads(data)

In [ ]:
%%time

json_data = ujson.dumps(ndb.database)
db = ujson.loads(json_data)

In [ ]:
%%time

json_data = json.dumps(ndb.database)
db = json.loads(json_data)

In [ ]:
%%time

db = copy.deepcopy(ndb.database)

In [ ]:
ndb.update()

In [ ]:
for ds in ndb.scenarios[0]["database"]:
    if ds["name"] == "market for petrol, low-sulfur, 20-year period" and ds["location"] == "WEU":
        for e in ds["exchanges"]:
            print(e["name"], e["amount"])

In [ ]:
ndb.write_db_to_brightway()

In [ ]:
import pandas as pd
fp = "/Users/romain/GitHub/carculator_utils/carculator_utils/data/emission_factors/car/EF_HBEFA42_exhaust.csv"
df = pd.read_csv(fp)

In [ ]:
df.loc[df["euro_class"]==6.2]

In [ ]:
import pandas as pd
fp = "/Users/romain/GitHub/premise/premise/data/iam_output_files/fleet_files/image_fleet_trucks.csv"
fp_out = "/Users/romain/GitHub/premise/premise/data/iam_output_files/fleet_files/image_fleet_trucks_.csv"
df = pd.read_csv(fp, sep=",")

In [ ]:
df

In [ ]:
df.groupby(["year", "region", "powertrain", "size"]).sum().to_csv(fp_out)

In [ ]:
from bw2io import ExcelImporter
import bw2data

bw2data.projects.set_current("new4")
fp = "/Users/romain/GitHub/premise/premise/data/additional_inventories/lci-pass_cars.xlsx"

i = ExcelImporter(fp)

In [ ]:
i.apply_strategies()

In [ ]:
bw2data.databases

In [ ]:
i.match_database(fields=["name", "reference product", "location"])
i.match_database("ecoinvent 3.7 cutoff", fields=["name", "reference product", "location"])
i.match_database("biosphere3", fields=["name", "categories",])

In [ ]:
i.statistics()

In [ ]:
list(i.unlinked)

In [ ]:
i.write_excel()

In [ ]:
ndb.write_db_to_brightway("test batt")

In [ ]:
ndb.update()

In [ ]:
ndb.write_db_to_brightway("test13")

In [ ]:
for ds in ndb.database:
    if ds["reference product"] == "biogas" and "market for" not in ds["name"]:
        print(ds["location"])
        for e in ds["exchanges"]:
            print(e["name"], e["amount"])
        print()

In [ ]:
ndb.write_db_to_brightway("test8")

In [ ]:
bw2data.

In [ ]:
FU = [
    {a: 1} for a in
    bw2data.Database("ecoinvent_cutoff_3.9_image_SSP2-Base_2050")
    if a["name"].startswith("electricity production") and a["unit"] == "kilowatt hour"
]

print(len(FU))

list_meth = [
    ('IPCC 2021', 'climate change', 'GWP 100a, incl. H'),
    ('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')
]
import bw2calc
bw2data.calculation_setups['multiLCA'] = {'inv': FU, 'ia': list_meth}
myMultiLCA = bw2calc.MultiLCA('multiLCA')
df_impact_exc = pd.DataFrame(columns = [", ".join(i) for i in list_meth],
     data = myMultiLCA.results,
     index=[k["name"] + ", " + k["location"] for i in FU for k in i]
)

In [ ]:
ndb.update_steel()

In [ ]:
ndb.update_biomass()

In [ ]:
ndb.update_electricity()

In [ ]:
ndb.write_db_to_simapro()

In [ ]:
for ds in ndb.scenarios[0]["database"]:
    if ds["name"].startswith("market group for electricity, low"):
        print(ds["name"], ds["reference product"], ds["location"])

In [ ]:
ndb.scenarios[1]["iam data"].carbon_capture_rate

In [ ]:
from collections import defaultdict
res = defaultdict(float)
for ds in ndb.scenarios[0]["database"]:
    if ds["reference product"]=="biogas":
        print(ds["location"], ds["name"])
        for e in ds["exchanges"]:
            print(e["name"], e["amount"], e.get("location"))
        print()



In [ ]:
from collections import defaultdict
res = defaultdict(float)
for ds in ndb.scenarios[1]["database"]:
    if ds["name"]=="market group for heat, district or industrial, natural gas":
        print(ds["location"], ds["name"])
        for e in ds["exchanges"]:
            print(e["name"], e["amount"], e.get("location"), e["type"])
        print()


In [ ]:
from collections import defaultdict
res = defaultdict(float)
for ds in ndb.scenarios[0]["database"]:
    if ds["name"]=="transport, freight, lorry, plugin diesel hybrid, 26t gross weight, 2020, EURO-VI, regional delivery" and ds["location"] == "ME":
        print(ds["location"], ds["name"])
        for e in ds["exchanges"]:
            print(e["name"], e["amount"], e.get("location"), e["type"])
        print()


In [ ]:
from premise.geomap import Geomap
geo = Geomap("remind")
geo.ecoinvent_to_iam_location("Europe, without Russia and Turkey")

In [ ]:
geo.iam_to_ecoinvent_location("EUR")

In [ ]:
geo.rev_additional_mappings

In [ ]:
from collections import defaultdict
res = defaultdict(float)
for ds in ndb.scenarios[0]["database"]:
    if ds["name"]=="pig iron production" and ds["location"] == "CHN":
        print(ds["location"], ds["name"])
        for e in ds["exchanges"]:
            print(e["name"], e["amount"])
        print(ds["comment"])
        print()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame.from_dict(res, orient="index").plot(kind="bar", figsize=(20, 5))
plt.ylabel("kg heavy fuel oil/kWh")
plt.savefig("electricity production, oil.png")

In [ ]:
ndb.scenarios[0]["iam data"].electricity_efficiencies.coords["variables"].values

In [ ]:
ndb.scenarios[0]["iam data"].electricity_efficiencies.sel(variables="Biomass IGCC CCS", year=2050)

In [ ]:
ndb.generate_change_report()

In [ ]:
ndb.scenarios[0]["iam data"].gas_markets.sel(region="World")

In [ ]:
ndb.scenarios[0].

In [ ]:
for ds in ndb.scenarios[0]["database"]:
    for e in ds["exchanges"]:
        if e["name"] == "market for platinum" and e["product"] == "platinum":
            print(ds["name"], ds["location"], e["name"], e["location"])

In [ ]:
import pprint
for ds in ndb.scenarios[0]["database"]:
    if ds["name"].lower() == "lithium carbonate, precipitated":
        print(ds["name"], ds["location"])

In [ ]:
from premise.activity_maps import InventorySet
inv = InventorySet(ndb.database, "3.9")

In [ ]:
inv.generate_metals_map()

In [ ]:
inv.generate_powerplant_map()

In [ ]:
inv.generate_daccs_map()

In [ ]:
ndb.write_db_to_brightway("test_metals")

In [ ]:
products = [
    'lightweighting, glider',
    'auxilliary battery inputs, for road vehicle',
    'battery cell, Li-ion, NMC622',
    'auxilliary electronics inputs, for fuel cell electric vehicle',
    'auxilliary material inputs, for fuel cell electric vehicle',
    'fuel cell stack, for fuel cell electric vehicle',
    'carbon fiber, weaved',
    'tank liner, polyethylene, high density',
    'transformer and rectifier unit, for electrolyser',
    'fuel tank, for diesel road vehicle',
    'fuel tank, for compressed hydrogen gas, 700bar',
    'fuel tank, for compressed hydrogen gas, 700bar',
    'fuel tank, for compressed hydrogen gas, 700bar',
    'fuel tank, for compressed natural gas, 200 bar',
    'charger, level 3, plugin, 200 kW, for electric vehicle',
    'gearbox, for lorry',
    'transmission, for lorry',
    'suspension, for lorry',
    'power electronics, for lorry',
    'auxilliary material inputs, for electric lorry',
    'retarder, for lorry',
    'cabin, for lorry',
    'frame, blanks and saddle, for lorry',
    'tires and wheels, for lorry',
    'auxilliary material inputs, for hydrid electric lorry',
    'exhaust system, for lorry',
    'internal combustion engine, for lorry',
    'lead acid battery, for lorry',
    'charger, level 3, with pantograph, 450 kW, for electric vehicle',
    'overhead lines, for electric vehicle',
    'battery cell, Li-ion, LTO',
    'charging station, 3kW',
    'battery cell,  Li-ion, LFP',
    'battery cell, Li-ion, NCA',
    'assembly operation, for lorry'
]

print(len(products))

acts = [a for a in bw.Database("brian") if a["reference product"] in products]

In [ ]:
len(acts)

In [ ]:
[a["reference product"] for a in acts]

In [ ]:
import bw2data
bw2io.BW2Package.export_obj(obj=bw.Database("brian"), filename="db_for_brian.bw2package")

In [ ]:
import bw2data
bw2io.BW2Package.import_file("db_for_brian.bw2package")

In [ ]:
fp = "/Users/romain/Library/Application Support/Brightway3/ei39.f1469ad5eb0a071dc60323f6e0fce980/output/lci-brian.xlsx"
import bw2io

i = bw2io.ExcelImporter(fp)
i.apply_strategies()

In [ ]:
ei_acts = list(set([(a["name"], a["reference product"]) for a in bw.Database("ecoinvent 3.9.1 cutoff")]))
print(len(ei_acts))

In [ ]:
for ds in i.data:
    if (ds["name"], ds["reference product"]) in ei_acts:
        i.data.remove(ds)

In [ ]:
for ds in i.data:
    if (ds["name"], ds["reference product"]) not in ei_acts:
        print(ds["name"])

In [ ]:
len(i.data)

In [ ]:
for ds in i.data:
    print(ds["name"])

In [ ]:
i.match_database(fields=["name", "unit", "location"])
i.match_database("ecoinvent 3.9.1 cutoff", fields=["name", "reference product", "location", "unit"])
i.match_database("biosphere3", fields=["name", "categories", "unit"])

In [ ]:
i.statistics()

In [ ]:
list(i.unlinked)[-1]

In [ ]:
bw.get_activity(('brian', '360c5e612a154ebb8cee2650bcc1bc75_copy1'))

In [ ]:
for f in bw.Database("biosphere3"):
    if f["code"] == "e030108f-2125-4bcb-a73b-ad72130fcca3":
        print(f)

In [ ]:
bw.projects.set_current("ei39")

In [ ]:
l = []

for f in bw.Database("biosphere3"):
    l.append([f["name"], f["categories"][0], f["categories"][1] if len(f["categories"])>1 else "unspecified", f["unit"], f["code"]])

In [ ]:
import csv

with open("flows_biosphere_39.csv", "w", newline="", ) as f:
    writer = csv.writer(f, delimiter=";")
    writer.writerows(l)

In [ ]:
ndb.scenarios[1]["iam data"].fuel_markets.sel(region="NEU").variables.values

In [ ]:
import numpy as np
for ds in ndb.scenarios[1]["database"]:
    if ds["name"] == ("carbon dioxide, captured from atmosphere, with a solvent-based direct air capture system, 1MtCO2, with waste heat, and grid electricity",
                            "carbon dioxide, captured from atmosphere"):
            print(ds["name"], ds["location"])

In [ ]:
for ds in ndb.scenarios[1]["database"]:
    if "carbon dioxide, captured" in ds["name"]:
        print(ds["name"], ds["location"])

In [ ]:
ndb.scenarios[0]["iam data"].fuel_markets.sel(region="WEU", year=2050)

In [ ]:
ndb.scenarios[0]["iam data"].fuel_markets.sel(region="WEU")

In [ ]:
ndb.update_electricity()

In [ ]:
ndb.scenarios[0]["iam data"].fuel_markets

In [ ]:
ndb.generate_scenario_report()

In [ ]:
ndb.generate_change_report()

In [ ]:
ndb.update_all()

In [ ]:
ndb.generate_scenario_report()

In [ ]:
ndb.generate_change_report()

In [ ]:
ndb.scenarios[0]["iam data"].gains_data_IAM

In [ ]:
ndb.update_all()

In [ ]:
ndb.write_db_to_brightway(["test_dac", "test_dac2"])

In [ ]:
from premise.geomap import Geomap

In [ ]:
Geomap()

In [ ]:
Geomap(model="remind").geo.keys()

In [ ]:
ndb.scenarios[0]["iam data"].carbon_capture_rate.sel(variables="steel")

In [ ]:
for ds in ndb.scenarios[0]["database"]:
    for e in ds["exchanges"]:
        if e["name"].startswith("CO2 capture, at cement production plant"):
            print(e["name"], e.get("location"), e["amount"])
            print(ds["name"])
            print("--------------------")